In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as ss

In [2]:
t2 = pd.read_csv('data/train.csv')

In [3]:
t = t2.head(100000)
features = [x for x in t.columns if (x.find('feature') >= 0)]


In [4]:
# ss.norm.rvs( loc=0, scale=1, size=(paths,len(t)-1) )[:,0]

Instead of looping through each row maybe use this as a activation funcion?

In [5]:
# Create x*w + b
class dense_layer():
    def __init__(self, n_input, n_neuron):
        self.neuron = n_neuron
        self.weight = 0.10 * np.zeros((n_input, n_neuron))
        self.bias = np.zeros((1, n_neuron))
        
    def forward(self, inputs):
        self.output = np.dot(inputs, self.weight) + self.bias

In [6]:
class OU_Activation():
    def SSR(actual, pred):
        ssr = np.sum(actual - pred)**2
        return ssr

    def ou_process(self, X, paths = 130, date_steps=16, kappa = 5, theta = -0.5, sigma = 0.5):
        N = len(X)
        self.n = N
        T_vec, dt = np.linspace(0, date_steps, N, retstep=True ) # Used for creating values that evenly spaced 
        
        self.t = theta
        self.k = kappa
        self.s = sigma
        self.dt = dt
        
        std_asy = np.sqrt( sigma**2 /(2*self.k) )   # asymptotic standard deviation
        self.W = ss.norm.rvs( loc=0, scale=1, size=(paths,N) )


        y_pred = []
        std_dt = np.sqrt( sigma**2 /(2*self.k) * (1-np.exp(-2*self.k*dt)) )
        for t in range(0,N):
            p = theta + np.exp(-self.k*dt) * (X[t]-theta) + std_dt * self.W[:,t]
            y_pred.append(p)
        y_pred = pd.DataFrame(y_pred)

        # Finding the minimum SSR
#         min_ssr = []
#         for i in range(len(y_pred.columns)):
#             ssr = SSR(y1, y_pred[i])
#             min_ssr.append(ssr)
#         min_ssr = pd.DataFrame(min_ssr)
#         min_ssr = min_ssr[min_ssr[0] == min_ssr[0].min()]

#         y_pred_final = []
#         # Filtering to the best fit
#         for i in range(0, N-1):
#             p = theta + np.exp(-self.k*dt) * (X[i]-theta) + std_dt * self.W[:, min_ssr.index[0]]
#             y_pred_final.append(p)
#         self.W = self.W[:, min_ssr.index[0]]
        
        return pd.DataFrame(y_pred)

#     def back_SSR(acutal, y_pred):
#         dssr = np.sum(-2(actual-pred))
#         return dssr

#     def backward_ou_process(theta, kappa, sigma):
#         ex = np.exp(-kappa*dt)
#         diff_ex = 1-ex
#         dthet = 1 + ex
#         dkappa = -dt * (X-theta) * ex + (W * np.abs(sigma)) * ( ( (2*dt*ex)/X) / (2**(3/2) * np.sqrt(diff_ex/X) ) )
#         dsgima = W * np.sqrt(diff_ex / X)
        
#     def backward_ou_process(kappa, dt):
#         dou = np.exp(-kappa * dt)
        
    
    
    def backwards(self, weights, a, p, bias, inputs, lr=.01):
        dou = np.exp(-self.k * self.dt)
        diff_ex = 1-dou
        #dW = np.sum(np.sum(-2*( a-p ) ) * np.sqrt( self.s**2 /(2*self.k) * (1-np.exp(-2*self.k*self.dt)) ))
        ou_and_bias = np.sum( np.sum(-2*(a-p)) * dou )
        dWeights = np.sum( np.sum(-2*(a-p)) * dou * inputs )

        dthet = np.sum(np.sum(-2*(a-p)) * (1 + dou))
        dsigma = np.sum(np.sum(-2*(a-p)) * ( (self.W.T * np.sqrt(diff_ex / inputs) * self.s) / (2 /np.abs(self.s) )) )
        
        dWeights = pd.DataFrame(dWeights).reset_index()
        dWeights = dWeights[[x for x in dWeights.columns if x != 'index']]
        for i in range(1,d.neuron):
            dWeights[i] = dWeights[0]
        
        #self.new_dW = self.W - (dW * lr)
        self.new_weights = pd.DataFrame(weights) - (dWeights * lr)
        self.new_bias = bias - (ou_and_bias * lr)
        self.new_theta = self.t - (dthet * lr)
        # self.new_kappa = self.k - (dkappa * lr)
        self.new_sigma = self.s - (dsigma * lr)
        del dWeights
    

In [7]:
y1 = t['resp']
t = t[features]

In [8]:
for i in t.columns:
    t[i] = t[i].fillna(t[i].mean())

In [9]:
ou = OU_Activation()

d = dense_layer(130,130)
d.forward(t)
SDE = ou.ou_process(d.output)

b = ou.backwards(weights=d.weight, a=y1,p=SDE[0], bias=d.bias, inputs=t)
output = np.dot(t, ou.new_weights) + ou.new_bias
y_pred = []

std_dt = np.sqrt( ou.new_sigma**2 /(2*ou.k) * (1-np.exp(-2*ou.k*ou.dt)) )
for t in range(0,ou.n):
    p = ou.new_theta + np.exp(-ou.k*ou.dt) * (output[t]-ou.new_theta) + std_dt * ou.W[:, t]
    y_pred.append(p)
#y_pred = pd.DataFrame(y_pred)

new_weights = ou.new_weights
new_bias = ou.new_bias
new_theta = ou.new_theta
new_sigma = ou.new_sigma

C:\Users\kenny\.jupyter\Anaconda\lib\site-packages\ipykernel_launcher.py:68: RuntimeWarning: invalid value encountered in sqrt


In [ ]:
for i in range(100):
    print(i)
    dou = np.exp(-ou.k * ou.dt)
    diff_ex = 1-dou
    #dW = np.sum(np.sum(-2*( a-p ) ) * np.sqrt( self.s**2 /(2*self.k) * (1-np.exp(-2*self.k*self.dt)) ))
    ou_and_bias = np.sum( np.sum(-2*(y1-y_pred['feature_0'])) * dou )
    dWeights = np.sum( np.sum(-2*(y1-y_pred['feature_0'])) * dou * t )

    dthet = np.sum(np.sum(-2*(y1-y_pred['feature_0'])) * (1 + dou))
    dsigma = np.sum( np.sum(-2*(y1-y_pred['feature_0'])) * ((ou.W.T * np.sqrt(diff_ex / t) * new_sigma) / (2 /np.abs(new_sigma) ) ))

    dWeights = pd.DataFrame(dWeights).reset_index()
    dWeights = dWeights[[x for x in dWeights.columns if x != 'index']]
    for i in range(1,d.neuron):
        dWeights[i] = dWeights[0]

    #self.new_dW = self.W - (dW * lr)
    new_weights = new_weights - (dWeights * lr)
    new_bias = new_bias - (ou_and_bias * lr)
    new_theta = new_theta - (dthet * lr)
    new_sigma = new_sigma - (dsigma * lr)
    
    output = np.dot(t, ou.new_weights) + ou.new_bias
    y_pred = []

    std_dt = np.sqrt( ou.new_sigma**2 /(2*ou.k) * (1-np.exp(-2*ou.k*ou.dt)) )
    for t in range(0,ou.n):
        p = new_theta + np.exp(-ou.k*ou.dt) * (output[t]-new_theta) + std_dt * ou.W[:, t]
        y_pred.append(p)
    y_pred = pd.DataFrame(y_pred)

In [10]:
y_pred.iloc[:, 0]

AttributeError: 'list' object has no attribute 'iloc'

In [ ]:
plt.plot(y_pred['feature_0'])

In [ ]:
ou.weight

In [ ]:
!conda install tensorflow==1.14